In [ ]:
# Reference : https://ampl.com/api/extra/python_quickstart.html

## Step 1: Path Setting

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import random
from operator import add

In [2]:
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
from amplpy import AMPL, Environment, DataFrame

## Step 2: Create an AMPL object

In [4]:
ampl = AMPL(Environment('C:\\Users\\user\\Desktop\\OR_Kung\\AMPL'))

## Step 3: Select the solver

In [5]:
ampl.setOption('solver','C:\\Users\\user\\Desktop\\OR_Kung\\AMPL\\cplex')  #'gurobi'

## Step 4: Define the model

In [6]:
# ampl.read('FairCHBF.mod')

In [7]:
ampl.eval('''
set I;
set J;

param B {J} >=0;   # benefit of doing the job
param C {J} >=0;   # cost of doing the job
param K {I} >=0;   # capacity of machine 

var X {I,J} >=0 binary;   # Decision var 1 # 其他問題寫"integer"
var MinBnft;              # Decision var 2

maximize Benefit: MinBnft;

s.t. Capacity {i in I        }: sum {j in J} C[j] * X[i,j] <= K[i];
s.t. ProdTime {        j in J}: sum {i in I}        X[i,j] <= 1;
s.t. MacBnft  {i in I        }: sum {j in J} B[j] * X[i,j] >= MinBnft;
s.t. Arrange  {i in I, j in J}:                     X[i,j] >= 0;

s.t. MinBenefit : MinBnft >=0;
''')

## Step 5: Define the initial data

In [11]:
# batch 要做成:
# [ num_machine, num_job, cost, benefit, capacity ]

it_n = 1

def batch (num_machine, num_job, relation_bc, capacity):
    
    m = num_machine
    n = num_job
    
    list_c1 = []    # 內含 100組 n個 job的 cost 組合
    list_b1 = []    # 內含 100組 n個 job的 bnft 組合
    list_k1 = []    # 內含 100組 由 n個 job 加總算出的 k
    
    for i in range(it_n):
        
        list_c2 = []   # 內含 n個 job的 cost
        list_b2 = []   # 內含 n個 job的 bnft
        
        for j in range(n):
            c = int(random.randint(0,50))
            list_c2.append(c)

            if   relation_bc == 'L':
                b = c

            elif relation_bc == 'X':
                b = c**2

            elif relation_bc == 'A':
                b = c**(1/2)

            elif relation_bc == 'R':
                b = random.randint(0,50)  
            list_b2.append(b)

        if   capacity =='N':
            k = 10000 # no capacity

        elif capacity =='L':
            k = sum(list_c2) / m

        elif capacity =='T':
            k = sum(list_c2) * (0.75) / m
    
        list_c1.append(list_c2)
        list_b1.append(list_b2)
        list_k1.append(k)
    
    return  m, n, list_c1, list_b1, list_k1

In [62]:
# Test OK! 不要在正式 run 100組時  print，會很慢 >"<
'''
instance = batch( 5, 20, 'A', 'T' )
print(instance[2])
print(instance[3])
'''

[[22, 48, 34, 16, 40, 30, 5, 48, 34, 17, 18, 5, 41, 26, 14, 12, 26, 45, 15, 35], [43, 35, 6, 18, 16, 31, 9, 8, 12, 35, 13, 23, 50, 48, 30, 48, 50, 35, 25, 29]]
[[4.69041575982343, 6.928203230275509, 5.830951894845301, 4.0, 6.324555320336759, 5.477225575051661, 2.23606797749979, 6.928203230275509, 5.830951894845301, 4.123105625617661, 4.242640687119285, 2.23606797749979, 6.4031242374328485, 5.0990195135927845, 3.7416573867739413, 3.4641016151377544, 5.0990195135927845, 6.708203932499369, 3.872983346207417, 5.916079783099616], [6.557438524302, 5.916079783099616, 2.449489742783178, 4.242640687119285, 4.0, 5.5677643628300215, 3.0, 2.8284271247461903, 3.4641016151377544, 5.916079783099616, 3.605551275463989, 4.795831523312719, 7.0710678118654755, 6.928203230275509, 5.477225575051661, 6.928203230275509, 7.0710678118654755, 5.916079783099616, 5.0, 5.385164807134504]]


In [35]:
machine_job  = [(3,10)]
capacity     = ['L'] # ,'T','N'
relation_bc  = ['L'] # ,'A','X','R'

## Step 6: Model Build-up & Solution

In [38]:
# 迭代 instance

MinBenefit_ijk_sn_1 = []
MinBenefit_ijk_sn_2 = []

for i in machine_job:
    
    num_machine = i[0]
    num_job     = i[1]

    # 印出編號，但從1開始算而不是從0，再讓 [1,2,3] -> [M1,M2,M3]

    machine1 = ['M']*num_machine
    job1     = ['J']*num_job
    machine2 = map(add, list(range(num_machine)), [1]*num_machine)
    job2     = map(add, list(range(num_job))    , [1]*num_job)
    machine2 = list( str(x) for x in machine2 )
    job2     = list( str(x) for x in job2     )
    machine  = list (map(lambda y, z: y + z, machine1, machine2))
    job      = list (map(lambda y, z: y + z, job1,     job2    ))

    # AMPL sets

    ampl.getSet('I').setValues(machine)
    ampl.getSet('J').setValues(job)
    
    for k in capacity:    
        for j in relation_bc:

            instance = batch( i[0], i[1], j, k )
            print(instance)    
            MinBenefit_1 = []
            MinBenefit_2 = []
            
            for p in range(it_n):      
                
                # ====== Model 1 ======= : 一組一組用solver算 IP (or linear relaxed LP)，每種senario要算 100組 再平均
                
                # amplpy DataFrame
                ampl.setData(DataFrame(
                index=[('J', job)], 
                columns=[
                    ('B', instance[3][p]), 
                    ('C', instance[2][p])
                ]
                ))

                # Pandas DataFrame
                df = pd.DataFrame({
                    'K': [ instance[4][p] ]*num_machine
                }, 
                    index = machine
                )
                ampl.setData(DataFrame.fromPandas(df))

                # Solve the problem
                ampl.solve()

                '''
                # Create a DataFrame with Decision variables
                Var = ampl.getVariable('X').getValues().toPandas()
                print(Var)
                '''

                # Display the objective value
                MinBnft_1 = ampl.getObjective('Benefit')
                MinBnft_1 = MinBnft_1.value()
                MinBenefit_1 = np.append(MinBenefit_1, MinBnft_1) # numpy.ndarray
                
                
                # ====== Model 2 ======= : 一組一組用 CHBF算，每種senario要算 100組 再平均
                
                # 將 job 按照 cost (workload) 排序

                list_c = instance[2][p]
                list_b = instance[3][p]

                list_cb = list(zip(list_c, list_b))        # [3,5]  [8,1]  -> [(3,8),(5,1)]
                s_cb    = sorted(list_cb, reverse = True)  # [(3,8),(5,1)] -> [(5,1),(3,8)]
                o_cb    = list(zip(*s_cb))                 # [(5,1),(3,8)] -> [(5,3),(1,8)]

                list_c  = list(o_cb[0])                    # [5,3]
                list_b  = list(o_cb[1])                    # [1,8]

                mach = []                 # machine
                ca = instance[4][p]

                for r in range(num_machine):

                    mach.append([])    # machine set中安排一台台 machine 被 append 進來
                    mach[r].append(0)  # 初始的 benefit和 是 0
                    mach[r].append(ca) # 初始的 capacity  是 ca

                for s in range(num_job):

                    mach = sorted(mach)                           # 按照 benefit和 由小而大排序
                    # print(machine)

                    for r in range(num_machine):                  # 由最小 benefit和 的先開始

                        if list_c[s] <= mach[r][1]:               # machine r 還有 capacity 餘裕

                            # print(mach[r])
                            mach[r][1] -= list_c[s]
                            mach[r][0] += list_b[s]
                            # print(mach[r])
                            # print('=================')
                            break
                            
                mach = sorted(mach)                               # 再一次 按照 benefit和 由小而大排序
                print(mach)
                MinBnft_2 = mach[0][0]                            # 將最小的 benefit和的 machine 取出來
                MinBenefit_2 = np.append(MinBenefit_2, MinBnft_2) # numpy.ndarray
                print(MinBenefit_2)

(3, 10, [[11, 43, 22, 27, 41, 23, 9, 45, 44, 11]], [[11, 43, 22, 27, 41, 23, 9, 45, 44, 11]], [92.0])
CPLEX 12.8.0.0: optimal integer solution; objective 88
234 MIP simplex iterations
43 branch-and-bound nodes
[[84, 8.0], [90, 2.0], [91, 1.0]]
[ 84.]


In [11]:
MinBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, MinBenefit_ijk_sn_2, MinBenefit_ijk_sn_1))
MinBenefit_ijk_sn_ratio

[0.95538003897835677,
 0.96233908518214184,
 0.98001575113211259,
 1.005522245522527,
 0.88425556031324715,
 0.95521810523887152,
 0.97967426831080073,
 0.99361198432103082,
 0.98696413100990199,
 0.67064294379296407,
 0.53177146646038231,
 1.0486770349424359]

In [ ]:
                
                
            # ====== Model 1 Objective value: ====== #
            
            list_M1          = list(MinBenefit_1)
            MinBenefit_100_1 = sum(list_M1) / len(list_M1)
            MinBenefit_ijk_sn_1.append(MinBenefit_100_1)   # i*j*k 種 senario 迭代
            
            # ====== Model 2 Objective value: ====== #
            
            list_M2         = list(MinBenefit_2)
            MinBenefit_100_2 = sum(list_M2) / len(list_M2)
            MinBenefit_ijk_sn_2.append(MinBenefit_100_2)   # i*j*k 種 senario 迭代
            
            print('=========================' + str(i) + str(j) + str(k) + '=========================') # 分隔           
            
print(MinBenefit_ijk_sn_1) # list
print(MinBenefit_ijk_sn_2) # list

MinBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, MinBenefit_ijk_sn_2, MinBenefit_ijk_sn_1))
MinBenefit_ijk_sn_ratio

## Other Function

In [22]:
# Increase the costs of beef and ham

cost = ampl.getParameter('cost')
cost.setValues({'BEEF': 5.01, 'HAM': 4.55})
print("Increased costs of beef and ham.")

# 再解一次

ampl.solve()
print("New objective value:", totalcost.value())

# 印出其中項目

Buy = ampl.getVariable('Buy')
print("Buy['BEEF'].val = {}".format(Buy['BEEF'].value()))

# Display the dual value of each constraint

diet = ampl.getConstraint('diet')
for nutr in nutrients:
    print("diet['{}'].dual = {}".format(nutr, diet[nutr].dual()))

Increased costs of beef and ham.
CPLEX 12.8.0.0: optimal solution; objective 144.0120033
0 simplex iterations (0 in phase I)
New objective value: 144.01200332502074
Buy['BEEF'].val = 5.226932668329172
diet['A'].dual = 0.0
diet['C'].dual = 0.0
diet['B1'].dual = 0.0
diet['B2'].dual = 0.7999285120532
diet['NA'].dual = -0.007531172069825434
diet['CAL'].dual = 0.0
